# Analyse der Relevanz zwischen Nachrichten und Suchtrends anhand Google-Trends

## Stufe 1: Importierung von den zu nutzenden Bibliotheken und Speicherung der Rohdaten

Es werden die Bibliotheken importiert, die in den sämtlichen Analysenschritten global verwendet werden. Im Anschluss dazu werden die angesammelten Rohdaten durch die API-Anfrage (News-Scraper) aufbereitet und in den folgenden Schritten verarbeitet.

<p><strong>* Importierung </strong></p>

In [9]:
import requests # zuständig für die API-Abfrage
import json # Um den (String-)Content der API-Antworten in JSON-Format umzuwandeln
import csv
import datetime

<p><strong>* Abfrage zur Speicherung der Rohdaten </strong></p>

In [10]:
#Konstanten
VALUE = "value"

URL_FORMAT = "http://h2920750.stratoserver.net:5000/data/{}/all"
NEWS = "news"
TRENDS = "trends"
LIST_ENTITIES = [NEWS, TRENDS]

# Treands-Daten werden täglich um 0 Uhr gecrawlt, weshalb deren Datum um einen Tag substrahiert werden soll.
def subtrace_one_day(data):
    DATE = "date"
    DATE_FORMAT = "%d/%m/%Y"
    substracted = datetime.datetime.strptime(data[DATE], DATE_FORMAT) - datetime.timedelta(days=1)
    data[DATE] = substracted.strftime(DATE_FORMAT)
    return data
    
list_data = [] # 0: news; 1: trends

for ENTITY in LIST_ENTITIES:
    response = requests.get(
        URL_FORMAT.format(ENTITY)
    )
    
    if(not response.ok):
        response.raise_for_status()
    
    data = response.json()[VALUE]
    
    if ENTITY is TRENDS:
        data = list(map(subtrace_one_day, data))
    
    list_data.append(
        data
    )
    
    file = open("./data/1raw/{}.csv".format(ENTITY), "w")
    dict_writer = csv.DictWriter(file, data[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(data)
    file.close()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Stufe 2: 

In [85]:
import pandas as pd
import spacy
import re
import os

In [86]:
nlp = spacy.load("de_core_news_sm")
news = pd.read_csv("./data/raw/news.csv")
trends = pd.read_csv("./data/raw/trends.csv")
news.head()

,name,date,headline,intro
0,Welt,01/06/2021,Mit seiner Mars-Landung kommt China den USA ge...,-
1,Welt,01/06/2021,Die Politik stürzt die Ärzte in einen kaum lös...,Ab nächster Woche sollen Kinder ab zwölf Jahre...
2,Welt,01/06/2021,"WHO benennt Virusvarianten um, um Diskriminier...",Die Weltgesundheitsorganisation will besorgnis...
3,Welt,01/06/2021,Die „Bundesnotbremse“ fällt weg – doch ein let...,Angela Merkel deutete vorige Woche eine möglic...
4,Welt,01/06/2021,"„Unerträgliche Arroganz“, „spaltet Gesellschaf...",Um insgesamt 16 Cent will Annalena Baerbock de...


In [90]:
def process_data(x):
    if x != "-": 
        # beseitigt vorhandene Sonderzeichen
        # wie z.B. , . ! ? etc.
        doc = nlp(re.sub('\W+',' ', x))
        processed = ""
        for tk in doc:
            # Stoppwörter werden aussortiert
            if (not tk.is_stop):
                # lemmatrisierte Wörter
                processed += tk.lemma_ + ";"
        if len(processed) > 0:    
            if len(processed) > 0 and processed[-1] == ";":
                processed = processed[:len(processed) - 1]
            if processed[0] == ";":
                processed = processed[1:]
        x = processed
    return x

process_data("Es ist ein schöner Text zum Testen")

'schön;Text;Testen'

In [ ]:
for param in ["headline", "intro"]:
    news[param] = news[param].apply(process_data)
    
news.head()

In [84]:
path = os.path.join(os.getcwd() + "/data/processed", "news.csv")
news.to_csv(path, encoding='utf-8')